# Fundamentos de Aprendizaje Automático


## Práctica 3


In [1]:
# Imports globales
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import compileall
import numpy as np
import time
import matplotlib.pyplot as plt

compileall.compile_dir('.',force=True,quiet=True)

from datos import Datos
from clasificadores.clasificador_regresion_logistica import ClasificadorRegresionLogistica
from preprocesamientoag.preprocesamiento_ag import PreprocesamientoAG
from plotModel import plotModel

%matplotlib inline

### Apartado 1 
#### Breve descripción de algunos detalles de la implementación


Pseudocódigo:
```
mientras fitness < max_fitness o generaciones < max_generaciones
    progenitores = seleccion_progenitores(poblacion)
    elite = seleccionar_elite(poblacion)
    sucesores = cruzar(progenitores)
    mutantes = mutar(sucesores)
    poblacion = mutantes + elite
```
Nótese que por optimización, varios pasos se han podido implementar aprovechando outputs de otras funciones. 

##### Mecanismo de cruce implementado

El mecanismo de cruce que hemos implementado es cruce uniforme, en el cual se elige un alelo de uno de los dos progenitores de manera aleatoria para cada alelo del sucesor. Dado que en nuestro caso la información de un individuo se representa en una cadena de bits, para generar un sucesor cruzando dos individuos simplemente rellenamos cada posición del array de bits con el bit de uno de los dos progenitores elegido alternativamente.

##### Mecanismo de mutacion

El mecanismo de mutación implementado consiste en recorrer el array de bits del individuo y alterar cada uno de ellos con una cierta probabilidad. La manera de alterarlos consiste en negar cada bit.

### Apartado 2 
Resultados  de  la  selección  de  atributos  mediante  el algoritmo genético para los siguientes casos:

In [2]:
clasificador = ClasificadorRegresionLogistica()
dataset=Datos('./conjunto_datos/wdbc.data')
max_fitness = 0.93

#### a) Tamaño de población = 50 ; Generaciones = 50 

In [3]:
ppag = PreprocesamientoAG(tamano_poblacion=50,max_generaciones=50, max_fitness=max_fitness)
ppag.seleccionarAtributos(dataset, clasificador,quiet=True)


selected = np.append(ppag.atributos_seleccionados.astype(bool),True)
print("Atributos seleccionados:")
print(np.asarray(dataset.nombreAtributos)[selected])
print("Puntuación en clasificacion:")
print(ppag.puntuacion)

Atributos seleccionados:
['Atributo2' 'Atributo3' 'Atributo5' 'Atributo7' 'Atributo10' 'Atributo12'
 'Atributo15' 'Atributo16' 'Atributo17' 'Atributo18' 'Atributo20'
 'Atributo21' 'Atributo24' 'Atributo26' 'Atributo28' 'Atributo29'
 'Atributo30' 'Diagnosis']
Puntuación en clasificacion:
0.937185929648


#### b) Tamaño de población = 50 ; Generaciones = 75 

In [4]:
ppag = PreprocesamientoAG(tamano_poblacion=50,max_generaciones=75, max_fitness=max_fitness)
ppag.seleccionarAtributos(dataset, clasificador,quiet=True)


selected = np.append(ppag.atributos_seleccionados.astype(bool),True)
print("Atributos seleccionados:")
print(np.asarray(dataset.nombreAtributos)[selected])
print("Puntuación en clasificacion:")
print(ppag.puntuacion)

Atributos seleccionados:
['Atributo1' 'Atributo2' 'Atributo3' 'Atributo4' 'Atributo5' 'Atributo9'
 'Atributo10' 'Atributo14' 'Atributo15' 'Atributo16' 'Atributo18'
 'Atributo19' 'Atributo20' 'Atributo22' 'Atributo23' 'Atributo24'
 'Atributo25' 'Atributo26' 'Atributo29' 'Diagnosis']
Puntuación en clasificacion:
0.93216080402


#### c) Tamaño de población = 75 ; Generaciones = 50  

In [5]:
ppag = PreprocesamientoAG(tamano_poblacion=75,max_generaciones=50, max_fitness=max_fitness)
ppag.seleccionarAtributos(dataset, clasificador,quiet=True)


selected = np.append(ppag.atributos_seleccionados.astype(bool),True)
print("Atributos seleccionados:")
print(np.asarray(dataset.nombreAtributos)[selected])
print("Puntuación en clasificacion:")
print(ppag.puntuacion)

Atributos seleccionados:
['Atributo1' 'Atributo2' 'Atributo3' 'Atributo7' 'Atributo11' 'Atributo13'
 'Atributo14' 'Atributo15' 'Atributo17' 'Atributo18' 'Atributo19'
 'Atributo20' 'Atributo24' 'Diagnosis']
Puntuación en clasificacion:
0.93216080402


#### d) Tamaño de población = 75 ; Generaciones = 75 

In [6]:
ppag = PreprocesamientoAG(tamano_poblacion=75,max_generaciones=75, max_fitness=max_fitness)
ppag.seleccionarAtributos(dataset, clasificador,quiet=True)


selected = np.append(ppag.atributos_seleccionados.astype(bool),True)
print("Atributos seleccionados:")
print(np.asarray(dataset.nombreAtributos)[selected])
print("Puntuación en clasificacion:")
print(ppag.puntuacion)

Atributos seleccionados:
['Atributo3' 'Atributo4' 'Atributo7' 'Atributo9' 'Atributo10' 'Atributo11'
 'Atributo12' 'Atributo13' 'Atributo16' 'Atributo17' 'Atributo18'
 'Atributo20' 'Atributo24' 'Atributo27' 'Atributo28' 'Atributo30'
 'Diagnosis']
Puntuación en clasificacion:
0.93216080402


### Apartado 3
En esta sección se exponen los resultados obtenidos con scikit

In [7]:
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn import preprocessing 
from sklearn.model_selection import GridSearchCV, train_test_split


encAtributos = preprocessing.OneHotEncoder(categorical_features=dataset.nominalAtributos[:-1],sparse=False)
X = encAtributos.fit_transform(dataset.datos[:,:-1])
Y = dataset.datos[:,-1] 
class_names = sorted(dataset.diccionarios[-1].keys())
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=6)

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X_train, Y_train)
clf = Pipeline([
 ('feature_selection', SelectFromModel(lsvc)),
 ('classification', LogisticRegression())
])
print('SelectFromModel')
clf.fit(X_test, Y_test)
mk = clf.named_steps['feature_selection'].get_support()
selected = np.append(mk, True)
print("Atributos seleccionados:")
print(np.asarray(dataset.nombreAtributos)[selected])
print("Puntuación en clasificacion:")
print(clf.score(X_test, Y_test))

SelectFromModel
Atributos seleccionados:
['Atributo3' 'Atributo4' 'Atributo14' 'Atributo24' 'Diagnosis']
Puntuación en clasificacion:
0.94298245614


Para el caso de SelectKBest utilizamos una busqueda en grid para recorrer todos los K menores que el número de atributos, para así quedarnos con la selección que se ajusta mejor y no depender del hiperparámetro K.

In [8]:
print('SelectKBest')
K = len(dataset.diccionarios) # Busca en grid de 1 hasta K
pipeline = Pipeline([('kbest', SelectKBest(f_classif)), ('lr', LogisticRegression())])
grid_search = GridSearchCV(pipeline, {'kbest__k': range(1,K), 'lr__C': np.logspace(-10, 10, 5)})
grid_search.fit(X_train, Y_train)
mk = clf.named_steps['feature_selection'].get_support()
selected = np.append(mk, True)
print("Atributos seleccionados:")
print(np.asarray(dataset.nombreAtributos)[selected])
print("Puntuación en clasificacion:")
print(clf.score(X_test, Y_test))

SelectKBest
Atributos seleccionados:
['Atributo3' 'Atributo4' 'Atributo14' 'Atributo24' 'Diagnosis']
Puntuación en clasificacion:
0.94298245614


### Apartado 4
Por último, en esta sección se mostrarán unas breves conclusiones.

En primer lugar destacar que el problema de selección de atributos mediante un algoritmo genético es un problema que se puede resolver de mejor manera sin mucha dificultad. Si nos fijamos, dados $k$ atributos, el espacio que queremos explorar tiene cardinalidad finita $2^k$ ($k$ bits que indican presencia o ausencia del atributo), por tanto, al ser un espacio relativamente pequeño (para $k=5$, hay 32 posibilidades, para $k=10$ hay 1024, el problema es el crecimiento exponencial), los algoritmos de scikit que exploran de manera codiciosa esto, resultan igual de satisfactorios y relativamente más eficientes para $k$ pequeño. 

Sobre los parámetros del algoritmo genético, las probabilidades de cruce o de mutación resultan muy importantes en el rendimiento del algoritmo genético. Los parámetros de tamaño de la población, máximo fitness y número de generaciones son los más importantes en relación al rendimiento, ya que el algoritmo es muy costoso computacionalmente y las condiciones de parada es relevante que se ajusten bien, dados los tiempos de ejecución que se dan. Para este problema, el número de generaciones y la población resultan innecesariamente altos, en general se ha observado que la población alcanza rápidamente un fitness en torno al 92-93% y se mantiene sin mejorar durante muchas generaciones.

Comparando ambas posibilidades (Scikit y nuestra implementación) y a la luz de los resultados, se preferirá el resultado de scikit que da un porcentaje mayor fitness y un subconjunto más pequeño de atributos. Nuestra implementación da un subconjunto mayor de atributos ya que ha eliminado los atributos suficientes como para que el fitness llegue al umbral pedido (o llegue al máximo número de generaciones).

Para conjuntos de datos con más variables, quizás el algoritmo genético sea menos costoso que este algoritmo de scikit, sin embargo, para este ejemplo, no termina de ser el caso.
